In [1]:
!apt-get update -qq
!apt-get install -y mysql-server > /dev/null
!service mysql start
!pip install -q mysql-connector-python

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
 * Starting MySQL database server mysqld
   ...done.


In [2]:
!mysql -uroot -prootpass -e "CREATE DATABASE IF NOT EXISTS my_database;"

mysql: [Warning] Using a password on the command line interface can be insecure.


In [20]:
# Cell 3: Create your tables
%%bash
set -e

mysql -uroot -prootpass <<'SQL'
USE my_database;

-- -------------------- CUSTOMER --------------------
CREATE TABLE IF NOT EXISTS Customer (
  customerId INT PRIMARY KEY,
  email VARCHAR(255) NOT NULL UNIQUE,
  custName VARCHAR(100) NOT NULL,
  age INT CHECK (age >= 0),
  gender VARCHAR(20)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

-- -------------------- EMAIL LIST --------------------
CREATE TABLE IF NOT EXISTS EmailList (
  listId INT PRIMARY KEY,
  listTitle VARCHAR(120) NOT NULL UNIQUE,
  createdDate DATE NOT NULL
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

-- -------------------- CUSTOMER EMAIL LIST --------------------
CREATE TABLE IF NOT EXISTS CustomerEmailList (
  customerId INT NOT NULL,
  listId INT NOT NULL,
  PRIMARY KEY (customerId, listId),
  FOREIGN KEY (customerId) REFERENCES Customer(customerId)
    ON UPDATE CASCADE ON DELETE CASCADE,
  FOREIGN KEY (listId) REFERENCES EmailList(listId)
    ON UPDATE CASCADE ON DELETE CASCADE
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

-- -------------------- EMPLOYEE --------------------
CREATE TABLE IF NOT EXISTS Employee (
  employeeId INT PRIMARY KEY,
  empName VARCHAR(100) NOT NULL,
  empTitle VARCHAR(100),
  tenure INT CHECK (tenure >= 0)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

-- -------------------- PURCHASE --------------------
CREATE TABLE IF NOT EXISTS Purchase (
  purchaseId INT PRIMARY KEY,
  customerId INT NOT NULL,
  purchDate DATE NOT NULL,
  quantity INT NOT NULL CHECK (quantity > 0),
  category VARCHAR(50) NOT NULL,
  amount DECIMAL(10,2) NOT NULL CHECK (amount >= 0),
  returned BOOLEAN NOT NULL DEFAULT FALSE,
  FOREIGN KEY (customerId) REFERENCES Customer(customerId)
    ON UPDATE CASCADE
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

-- -------------------- RETURNS --------------------
CREATE TABLE IF NOT EXISTS Returns (
  returnId INT PRIMARY KEY,
  purchaseId INT NOT NULL UNIQUE,
  employeeId INT NOT NULL,
  returnDate DATE NOT NULL,
  csat_score DECIMAL(3,2) CHECK (csat_score >= 0 AND csat_score <= 5),
  CONSTRAINT fk_returns_purchase
    FOREIGN KEY (purchaseId) REFERENCES Purchase(purchaseId)
    ON UPDATE CASCADE,
  CONSTRAINT fk_returns_employee
    FOREIGN KEY (employeeId) REFERENCES Employee(employeeId)
    ON UPDATE CASCADE
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

-- Create useful indexes
CREATE INDEX idx_purchase_customer ON Purchase(customerId);
CREATE INDEX idx_purchase_date ON Purchase(purchDate);
CREATE INDEX idx_returns_employee ON Returns(employeeId);
CREATE INDEX idx_returns_date ON Returns(returnDate);
SQL

mysql: [Warning] Using a password on the command line interface can be insecure.


In [21]:
!mysql -uroot -prootpass -e "USE my_database; SHOW TABLES;"

mysql: [Warning] Using a password on the command line interface can be insecure.
+-----------------------+
| Tables_in_my_database |
+-----------------------+
| Customer              |
| CustomerEmailList     |
| EmailList             |
| Employee              |
| Purchase              |
| Returns               |
+-----------------------+


In [5]:
# Cell 5: Upload and process your CSV
import pandas as pd
import mysql.connector
from google.colab import files
import numpy as np

# Upload your CSV
print("Upload your CSV file:")
uploaded = files.upload()
csv_filename = list(uploaded.keys())[0]

Upload your CSV file:


Saving purchasing_2023.csv to purchasing_2023 (1).csv


In [27]:
# Clear all data from tables
!mysql -uroot -prootpass -e "USE my_database; SET FOREIGN_KEY_CHECKS = 0; TRUNCATE TABLE Returns; TRUNCATE TABLE Purchase; TRUNCATE TABLE CustomerEmailList; TRUNCATE TABLE Employee; TRUNCATE TABLE EmailList; TRUNCATE TABLE Customer; SET FOREIGN_KEY_CHECKS = 1;"
print("✓ All tables cleared")

mysql: [Warning] Using a password on the command line interface can be insecure.
✓ All tables cleared


In [29]:
# Read the CSV
print(f"\nReading {csv_filename}...")
df = pd.read_csv(csv_filename)

print(f"Loaded {len(df)} rows")
print(f"Columns: {df.columns.tolist()}")

# ============== DATA TRANSFORMATION ==============

# 1. Handle N/A values FIRST - replace 'n/a', 'N/A', empty strings with actual NaN
df = df.replace(['n/a', 'N/A', 'na', 'NA', ''], np.nan)

# 2. Generate email from customer_name - make it unique with customer_id
df['customer_name_clean'] = df['customer_name'].fillna('Unknown Customer')
# Always include customer_id to guarantee uniqueness
df['email'] = df.apply(lambda row:
    row['customer_name_clean'].strip().replace(' ', '.').lower() + f".{int(row['customer_id'])}@example.com",
    axis=1)

# Make email unique by appending customer_id if duplicate
df['email'] = df.apply(lambda row: f"customer.{int(row['customer_id'])}@example.com"
                        if pd.isna(row['customer_name'])
                        else row['email'], axis=1)

# 3. Convert dates to proper format (handle MM/DD/YYYY HH:MM format)
print("\nConverting dates...")
df['purchase_date'] = pd.to_datetime(df['purchase_date'], format='%m/%d/%Y %H:%M', errors='coerce')
df['call_date'] = pd.to_datetime(df['call_date'], format='%m/%d/%Y %H:%M', errors='coerce')

print(f"Valid purchase dates: {df['purchase_date'].notna().sum()}")
print(f"Valid call dates: {df['call_date'].notna().sum()}")

# 4. Create unique IDs for entities
# Customer IDs - use existing customer_id
df['customerId'] = df['customer_id']

# Email List IDs - create based on call_category
emaillist_df = df[df['call_category'].notna()][['call_category']].drop_duplicates()
emaillist_df['listId'] = range(1, len(emaillist_df) + 1)
emaillist_df['createdDate'] = pd.Timestamp.today().date()
emaillist_df.rename(columns={'call_category': 'listTitle'}, inplace=True)

# Employee IDs - collect ALL employees (agents, supervisors, managers)
employees_list = []

# Agents with their employment length (as text)
agents = df[df['agent'].notna()][['agent', 'employment_length']].drop_duplicates()
for _, row in agents.iterrows():
    tenure_text = str(row['employment_length']) if pd.notna(row['employment_length']) else None
    employees_list.append({
        'empName': row['agent'],
        'empTitle': 'Agent',
        'tenure': tenure_text
    })

# Supervisors (no tenure)
supervisors = df[df['supervisor'].notna()]['supervisor'].unique()
for supervisor in supervisors:
    if not any(emp['empName'] == supervisor for emp in employees_list):
        employees_list.append({
            'empName': supervisor,
            'empTitle': 'Supervisor',
            'tenure': None
        })

# Managers (no tenure)
managers = df[df['manager'].notna()]['manager'].unique()
for manager in managers:
    if not any(emp['empName'] == manager for emp in employees_list):
        employees_list.append({
            'empName': manager,
            'empTitle': 'Manager',
            'tenure': None
        })

# Create employee dataframe
employee_df = pd.DataFrame(employees_list)
employee_df['employeeId'] = range(1, len(employee_df) + 1)

print(f"\nFound {len(employee_df)} unique employees:")
print(f"  - {len(employee_df[employee_df['empTitle']=='Agent'])} Agents")
print(f"  - {len(employee_df[employee_df['empTitle']=='Supervisor'])} Supervisors")
print(f"  - {len(employee_df[employee_df['empTitle']=='Manager'])} Managers")

# Purchase IDs - auto-increment
df['purchaseId'] = range(1, len(df) + 1)

# Return IDs - only for rows with returns
df['return_clean'] = df['return'].astype(str).str.lower().str.strip()
return_mask = (
    df['return'].notna() &
    (df['return_clean'] != 'nan') &
    (df['return_clean'] != 'n/a') &
    (df['return_clean'] != 'false') &
    (df['return_clean'] != '0') &
    (df['return_clean'] != '')
)

df['returnId'] = np.nan
df.loc[return_mask, 'returnId'] = range(1, return_mask.sum() + 1)

print(f"\nRows with returns: {return_mask.sum()}")

# 5. Map employee names to IDs (using agent column for the foreign key)
agent_to_id = dict(zip(employee_df['empName'], employee_df['employeeId']))
df['employeeId'] = df['agent'].map(agent_to_id)

# 6. Map call categories to list IDs
category_to_listid = dict(zip(emaillist_df['listTitle'], emaillist_df['listId']))
df['listId'] = df['call_category'].map(category_to_listid)

# 7. Convert returned column to boolean
df['returned'] = return_mask

# ============== PREPARE TABLE DATA ==============

# Customer table - include ALL unique customer_ids from the data
# Customer table - use BEST row for each customer (prefer rows with real data)
# Sort by: 1) non-null names first, 2) non-null age, 3) non-null gender
customer_temp = df[['customerId', 'email', 'customer_name_clean', 'customer_age', 'customer_gender']].copy()
customer_temp['name_not_null'] = customer_temp['customer_name_clean'].notna() & (customer_temp['customer_name_clean'] != 'Unknown Customer')
customer_temp['age_not_null'] = customer_temp['customer_age'].notna()
customer_temp['gender_not_null'] = customer_temp['customer_gender'].notna()

# Sort to get best rows first, then drop duplicates
customer_df = customer_temp.sort_values(['customerId', 'name_not_null', 'age_not_null', 'gender_not_null'],
                                         ascending=[True, False, False, False]).drop_duplicates(subset=['customerId'], keep='first')
customer_df = customer_df[['customerId', 'email', 'customer_name_clean', 'customer_age', 'customer_gender']]
customer_df.columns = ['customerId', 'email', 'custName', 'age', 'gender']

# Fill missing ages with a default or None
customer_df['age'] = customer_df['age'].fillna(-1)  # Use -1 to indicate unknown, or use None

print(f"\nTotal unique customers: {len(customer_df)}")
print(f"Customers with missing names: {(customer_df['custName'] == 'Unknown Customer').sum()}")

# EmailList table - already created above

# CustomerEmailList table - only for customers who called
customer_emaillist_df = df[df['listId'].notna()][['customerId', 'listId']].drop_duplicates()

# Employee table - already created above

# Purchase table - ALL purchases
purchase_df = df[['purchaseId', 'customerId', 'purchase_date', 'product_quantity',
                   'product_category', 'purchase_amount', 'returned']].copy()
purchase_df.columns = ['purchaseId', 'customerId', 'purchDate', 'quantity',
                       'category', 'amount', 'returned']

# Returns table - only rows with actual returns
returns_df = df[df['returnId'].notna()][['returnId', 'purchaseId', 'employeeId',
                                          'call_date', 'csat_score']].copy()
returns_df.columns = ['returnId', 'purchaseId', 'employeeId', 'returnDate', 'csat_score']

print(f"\nData prepared:")
print(f"  - Customers: {len(customer_df)}")
print(f"  - Email Lists: {len(emaillist_df)}")
print(f"  - Customer-EmailList relationships: {len(customer_emaillist_df)}")
print(f"  - Employees: {len(employee_df)}")
print(f"  - Purchases: {len(purchase_df)}")
print(f"  - Returns: {len(returns_df)}")

# DEBUG: Check what customer IDs are missing
print("\n=== DEBUGGING ===")
all_purchase_customers = set(df['customerId'].unique())
customers_in_table = set(customer_df['customerId'].unique())
missing_customers = all_purchase_customers - customers_in_table

print(f"Total unique customer IDs in purchases: {len(all_purchase_customers)}")
print(f"Customers in customer_df: {len(customers_in_table)}")
print(f"Missing customers: {len(missing_customers)}")
if len(missing_customers) > 0:
    print(f"Sample missing customer IDs: {list(missing_customers)[:10]}")
    # Check the raw data for one missing customer
    sample_missing = list(missing_customers)[0]
    print(f"\nRaw data for missing customer {sample_missing}:")
    print(df[df['customerId'] == sample_missing][['customerId', 'customer_name', 'customer_age', 'customer_gender']].head())

# ============== INSERT INTO DATABASE ==============

conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='rootpass',
    database='my_database'
)
cursor = conn.cursor()

print("\n" + "="*50)
print("LOADING DATA INTO TABLES")
print("="*50)

# 1. Load Customer
print("\n1. Loading Customer table...")
inserted = 0
errors = 0
for _, row in customer_df.iterrows():
    try:
        age_val = int(row['age']) if row['age'] != -1 else None
        cursor.execute(
            "INSERT IGNORE INTO Customer (customerId, email, custName, age, gender) VALUES (%s, %s, %s, %s, %s)",
            (int(row['customerId']),
             row['email'],
             row['custName'],
             age_val,
             row['gender'] if pd.notna(row['gender']) else None)
        )
        inserted += 1
    except Exception as e:
        errors += 1
        if errors <= 3:
            print(f"Error: {e}")
            print(f"Row: {row.to_dict()}")
conn.commit()
print(f"✓ Inserted {inserted} customers")

# 2. Load EmailList
print("\n2. Loading EmailList table...")
inserted = 0
for _, row in emaillist_df.iterrows():
    try:
        cursor.execute(
            "INSERT IGNORE INTO EmailList (listId, listTitle, createdDate) VALUES (%s, %s, %s)",
            (int(row['listId']), row['listTitle'], row['createdDate'])
        )
        inserted += 1
    except Exception as e:
        print(f"Error: {e}")
conn.commit()
print(f"✓ Inserted {inserted} email lists")

# 3. Load Employee
print("\n3. Loading Employee table...")
inserted = 0
for _, row in employee_df.iterrows():
    try:
        cursor.execute(
            "INSERT IGNORE INTO Employee (employeeId, empName, empTitle, tenure) VALUES (%s, %s, %s, %s)",
            (int(row['employeeId']),
             row['empName'],
             row['empTitle'],
             row['tenure'] if pd.notna(row['tenure']) else None)
        )
        inserted += 1
    except Exception as e:
        print(f"Error: {e}")
conn.commit()
print(f"✓ Inserted {inserted} employees")

# 4. Load Purchase
print("\n4. Loading Purchase table...")
inserted = 0
errors = 0
for idx, row in purchase_df.iterrows():
    try:
        # Convert date to string format
        purch_date = row['purchDate'].strftime('%Y-%m-%d') if pd.notna(row['purchDate']) else None

        if purch_date is None:
            errors += 1
            continue

        cursor.execute(
            "INSERT INTO Purchase (purchaseId, customerId, purchDate, quantity, category, amount, returned) VALUES (%s, %s, %s, %s, %s, %s, %s)",
            (int(row['purchaseId']),
             int(row['customerId']),
             purch_date,
             int(row['quantity']) if pd.notna(row['quantity']) else 1,
             row['category'] if pd.notna(row['category']) else 'Unknown',
             float(row['amount']) if pd.notna(row['amount']) else 0.0,
             bool(row['returned']))
        )
        inserted += 1
    except Exception as e:
        errors += 1
        if errors <= 5:
            print(f"Error on row {idx}: {e}")
            print(f"Row data: {row.to_dict()}")
conn.commit()
print(f"✓ Inserted {inserted} purchases ({errors} errors)")

# 5. Load CustomerEmailList
print("\n5. Loading CustomerEmailList table...")
inserted = 0
for _, row in customer_emaillist_df.iterrows():
    try:
        cursor.execute(
            "INSERT IGNORE INTO CustomerEmailList (customerId, listId) VALUES (%s, %s)",
            (int(row['customerId']), int(row['listId']))
        )
        inserted += 1
    except Exception as e:
        print(f"Error: {e}")
conn.commit()
print(f"✓ Inserted {inserted} customer-emaillist relationships")

# 6. Load Returns
print("\n6. Loading Returns table...")
inserted = 0
errors = 0
for idx, row in returns_df.iterrows():
    try:
        return_date = row['returnDate'].strftime('%Y-%m-%d') if pd.notna(row['returnDate']) else None

        if return_date is None:
            errors += 1
            continue

        cursor.execute(
            "INSERT INTO Returns (returnId, purchaseId, employeeId, returnDate, csat_score) VALUES (%s, %s, %s, %s, %s)",
            (int(row['returnId']),
             int(row['purchaseId']),
             int(row['employeeId']) if pd.notna(row['employeeId']) else None,
             return_date,
             float(row['csat_score']) if pd.notna(row['csat_score']) else None)
        )
        inserted += 1
    except Exception as e:
        errors += 1
        if errors <= 5:
            print(f"Error on row {idx}: {e}")
            print(f"Row: {row.to_dict()}")
conn.commit()
print(f"✓ Inserted {inserted} returns ({errors} errors)")

cursor.close()
conn.close()

print("\n" + "="*50)
print("✓ DATA LOADING COMPLETE!")
print("="*50)


Reading purchasing_2023 (1).csv...
Loaded 22801 rows
Columns: ['customer_id', 'purchase_date', 'product_category', 'product_quantity', 'purchase_amount', 'return', 'customer_name', 'customer_age', 'customer_gender', 'call_category', 'call_reason', 'call_date', 'agent', 'supervisor', 'manager', 'employment_length', 'csat_score']

Converting dates...
Valid purchase dates: 22801
Valid call dates: 2361

Found 957 unique employees:
  - 911 Agents
  - 40 Supervisors
  - 6 Managers

Rows with returns: 2361

Total unique customers: 18321
Customers with missing names: 0

Data prepared:
  - Customers: 18321
  - Email Lists: 1
  - Customer-EmailList relationships: 2315
  - Employees: 957
  - Purchases: 22801
  - Returns: 2361

=== DEBUGGING ===
Total unique customer IDs in purchases: 18321
Customers in customer_df: 18321
Missing customers: 0

LOADING DATA INTO TABLES

1. Loading Customer table...
✓ Inserted 18321 customers

2. Loading EmailList table...
✓ Inserted 1 email lists

3. Loading Emplo

In [30]:
# Cell 6: Verify your data
%%bash
mysql -uroot -prootpass <<'SQL'
USE my_database;
SELECT 'Customer' as TableName, COUNT(*) as RowCount FROM Customer
UNION ALL SELECT 'EmailList', COUNT(*) FROM EmailList
UNION ALL SELECT 'CustomerEmailList', COUNT(*) FROM CustomerEmailList
UNION ALL SELECT 'Employee', COUNT(*) FROM Employee
UNION ALL SELECT 'Purchase', COUNT(*) FROM Purchase
UNION ALL SELECT 'Returns', COUNT(*) FROM Returns;
SQL

TableName	RowCount
Customer	18321
EmailList	1
CustomerEmailList	2315
Employee	957
Purchase	22801
Returns	2361


mysql: [Warning] Using a password on the command line interface can be insecure.
